In [122]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import librosa
import librosa.display
import os
import tensorflow as tf

In [123]:
#music_genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
data_dir="../input/gtzan-dataset-music-genre-classification/Data/images_original"
music_genres = ['blues', 'classical', 'jazz' , 'metal', 'pop']
batch_size = 50
img_height = 288
img_width = 432

In [124]:
import keras

In [125]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image_dataset_from_directory

In [126]:
training_set = image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode='categorical',
  color_mode='rgb',
  shuffle=True,
)

Found 999 files belonging to 10 classes.
Using 700 files for training.


In [127]:
testing_set = image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode='categorical',
  color_mode='rgb',
  shuffle=True,
)

Found 999 files belonging to 10 classes.
Using 299 files for validation.


In [128]:
AUTOTUNE = tf.data.AUTOTUNE

training_set = training_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
testing_set = testing_set.cache().prefetch(buffer_size=AUTOTUNE)

In [129]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
from keras.layers import Dropout, BatchNormalization , ConvLSTM2D , TimeDistributed
from keras.layers.experimental.preprocessing import Rescaling
from keras import regularizers

In [130]:
num_classes = 10
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])


In [131]:
 model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 286, 430, 32)      896       
_________________________________________________________________
batch_normalization_15 (Batc (None, 286, 430, 32)      128       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 143, 215, 32)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 141, 213, 64)      18496     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 70, 106, 64)       0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 70, 106, 64)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 68, 104, 128)     

In [132]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=5, 
                                            verbose=1, 
                                            )
callbacks = [earlystop, learning_rate_reduction]

In [133]:
model.fit_generator(
    generator=training_set,
    validation_data=testing_set,
    verbose=1,
    callbacks=callbacks,
    epochs=120,
)

Epoch 1/120
14/14 [==============================] - 18s 944ms/step - loss: 142.6624 - accuracy: 0.1317 - val_loss: 378.6062 - val_accuracy: 0.1371
Epoch 2/120
14/14 [==============================] - 10s 742ms/step - loss: 50.8140 - accuracy: 0.2706 - val_loss: 174.6072 - val_accuracy: 0.2007
Epoch 3/120
14/14 [==============================] - 10s 742ms/step - loss: 17.9189 - accuracy: 0.4381 - val_loss: 154.3190 - val_accuracy: 0.1472
Epoch 4/120
14/14 [==============================] - 10s 743ms/step - loss: 8.3341 - accuracy: 0.6035 - val_loss: 109.5384 - val_accuracy: 0.2174
Epoch 5/120
14/14 [==============================] - 10s 742ms/step - loss: 3.3305 - accuracy: 0.7325 - val_loss: 73.8291 - val_accuracy: 0.2274
Epoch 6/120
14/14 [==============================] - 10s 741ms/step - loss: 1.2807 - accuracy: 0.8419 - val_loss: 64.7140 - val_accuracy: 0.1639
Epoch 7/120
14/14 [==============================] - 10s 742ms/step - loss: 1.0085 - accuracy: 0.8685 - val_loss: 34.3875 

In [134]:
model.save('./MSD_CNN_10.h5')